# From ChatGPT

In [ ]:
import RPi.GPIO as GPIO
import time

# Use board pin numbering
GPIO.setmode(GPIO.BOARD)

# Setup the pins the sensor is connected to
RED_PIN = 7
GREEN_PIN = 11
BLUE_PIN = 13
GPIO.setup(RED_PIN, GPIO.IN)
GPIO.setup(GREEN_PIN, GPIO.IN)
GPIO.setup(BLUE_PIN, GPIO.IN)

# Create a function to measure the sensor's frequency
def measure_frequency(pin):
    start_time = time.time()
    start_state = GPIO.input(pin)
    flips = 0

    while True:
        state = GPIO.input(pin)
        if state != start_state:
            flips += 1
            start_state = state
        if flips > 1000:
            break

    end_time = time.time()
    return flips / (end_time - start_time)

# Use the sensor
while True:
    red = measure_frequency(RED_PIN)
    green = measure_frequency(GREEN_PIN)
    blue = measure_frequency(BLUE_PIN)
    print(f"Red: {red}, Green: {green}, Blue: {blue}")
    time.sleep(1)

In [ ]:
import time
import RPi.GPIO as GPIO

# Define GPIO pins
S0 = 5
S1 = 6
S2 = 23
S3 = 24
OUT = 25

# Set up GPIO
GPIO.setmode(GPIO.BCM)
GPIO.setup(S0, GPIO.OUT)
GPIO.setup(S1, GPIO.OUT)
GPIO.setup(S2, GPIO.OUT)
GPIO.setup(S3, GPIO.OUT)
GPIO.setup(OUT, GPIO.IN, pull_up_down=GPIO.PUD_UP)

# Set frequency scaling to 20%
GPIO.output(S0, GPIO.HIGH)
GPIO.output(S1, GPIO.LOW)

while True:
    GPIO.output(S2, GPIO.LOW)
    GPIO.output(S3, GPIO.LOW)
    # Measure the output frequency for RED
    time.sleep(0.1)  # Wait for a bit to let the output stabilize
    start_time = time.time()
    for impulse_count in range(NUM_CYCLES):
        GPIO.wait_for_edge(OUT, GPIO.FALLING)
    red_duration = time.time() - start_time  # Measure the time of NUM_CYCLES falling edges

    # Repeat the above process for GREEN (S2=HIGH, S3=HIGH) and BLUE (S2=LOW, S3=HIGH) colors
    # Then, you can calculate the frequency of each color and convert it to RGB values

    # The rest of your color to pH conversion code goes here

    time.sleep(1)  # Wait for a second before repeating the loop

In [ ]:
import time
import RPi.GPIO as GPIO

class TCS3200:
    # Define GPIO pins
    def __init__(self, S0, S1, S2, S3, OUT, LED):
        self.S0 = S0
        self.S1 = S1
        self.S2 = S2
        self.S3 = S3
        self.OUT = OUT
        self.LED = LED  # Added LED pin
        self.setup_gpio()

    def setup_gpio(self):
        # Set up GPIO
        GPIO.setmode(GPIO.BCM)
        GPIO.setup(self.S0, GPIO.OUT)
        GPIO.setup(self.S1, GPIO.OUT)
        GPIO.setup(self.S2, GPIO.OUT)
        GPIO.setup(self.S3, GPIO.OUT)
        GPIO.setup(self.OUT, GPIO.IN, pull_up_down=GPIO.PUD_UP)
        GPIO.setup(self.LED, GPIO.OUT)  # Setup LED as output

        # Set frequency scaling to 20%
        GPIO.output(self.S0, GPIO.HIGH)
        GPIO.output(self.S1, GPIO.LOW)
        return

    def led_on(self):  # Method to turn LED on
        GPIO.output(self.LED, GPIO.HIGH)
        return

    def led_off(self):  # Method to turn LED off
        GPIO.output(self.LED, GPIO.LOW)
        return

    def read_color(self):
        colors = ['RED', 'GREEN', 'BLUE']
        filters = [(GPIO.LOW, GPIO.LOW), (GPIO.HIGH, GPIO.HIGH), (GPIO.LOW, GPIO.HIGH)]
        color_freq = {}

        for color, (S2_state, S3_state) in zip(colors, filters):
            GPIO.output(self.S2, S2_state)
            GPIO.output(self.S3, S3_state)
            start_time = time.time()
            for impulse_count in range(100):
                GPIO.wait_for_edge(self.OUT, GPIO.FALLING)
            duration = time.time() - start_time
            frequency = 100 / duration
            color_freq[color] = frequency

        # Normalize frequency values for RGB to [0, 255]
        max_freq = max(color_freq.values())
        for color in colors:
            color_freq[color] = int((color_freq[color] / max_freq) * 255)

        print(f"RGB({color_freq['RED']}, {color_freq['GREEN']}, {color_freq['BLUE']})")

        time.sleep(1)   # Wait for a second before repeating the loop
        return

if __name__ == '__main__':
    print("raw_rgb.py is running.")
    try:
        sensor = TCS3200(S0=5, S1=6, S2=23, S3=24, OUT=25, LED=18)  # Added LED pin number here
        print("GPIO pins are setup.")
        #sensor.led_on()  # Turn on LED before reading colors
        while True:
            sensor.read_color()
    except KeyboardInterrupt:
        print("Keyboard interrupted")
        #sensor.led_off()  # Turn off LED before cleaning up GPIO and exiting
        GPIO.cleanup()
        time.sleep(1)
        quit()


# From https://www.electronicshub.org/raspberry-pi-color-sensor-tutorial/

In [ ]:
import RPi.GPIO as GPIO
import time



s2 = 23
s3 = 24
signal = 25
NUM_CYCLES = 10


def setup():
  GPIO.setmode(GPIO.BCM)
  GPIO.setup(signal,GPIO.IN, pull_up_down=GPIO.PUD_UP)
  GPIO.setup(s2,GPIO.OUT)
  GPIO.setup(s3,GPIO.OUT)
  print("\n")





def loop():
  temp = 1
  while(1):

    GPIO.output(s2,GPIO.LOW)
    GPIO.output(s3,GPIO.LOW)
    time.sleep(0.3)
    start = time.time()
    for impulse_count in range(NUM_CYCLES):
      GPIO.wait_for_edge(signal, GPIO.FALLING)
    duration = time.time() - start      #seconds to run for loop
    red  = NUM_CYCLES / duration   #in Hz
    print("red value - ",red)

    GPIO.output(s2,GPIO.LOW)
    GPIO.output(s3,GPIO.HIGH)
    time.sleep(0.3)
    start = time.time()
    for impulse_count in range(NUM_CYCLES):
      GPIO.wait_for_edge(signal, GPIO.FALLING)
    duration = time.time() - start
    blue = NUM_CYCLES / duration
    print("blue value - ",blue)

    GPIO.output(s2,GPIO.HIGH)
    GPIO.output(s3,GPIO.HIGH)
    time.sleep(0.3)
    start = time.time()
    for impulse_count in range(NUM_CYCLES):
      GPIO.wait_for_edge(signal, GPIO.FALLING)
    duration = time.time() - start
    green = NUM_CYCLES / duration
    print("green value - ",green)
    time.sleep(2)


def endprogram():
    GPIO.cleanup()

if __name__=='__main__':

    setup()

    try:
        loop()

    except KeyboardInterrupt:
        endprogram()

# ME

In [ ]:
import RPi.GPIO as GPIO
import time


